**Solve XOR Classification using RBF Network**

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Layer, Dense, Flatten
from keras.models import Model
from keras.losses import binary_crossentropy
from sklearn.cluster import KMeans

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        kmeans = KMeans(n_clusters=self.units, random_state=0).fit(X) 
        self.mu = self.add_weight(name='mu',
                                  shape=(self.units, input_shape[1]),
                                  initializer=tf.constant_initializer(kmeans.cluster_centers_),
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = tf.expand_dims(inputs, axis=1) - self.mu
        l2 = tf.reduce_sum(tf.square(diff), axis=2) 
        res = tf.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([0, 1, 1, 0], dtype=np.float32)

input_layer = Input(shape=(2,))
x = Flatten()(input_layer)
x = RBFLayer(4, 0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='rmsprop', loss=binary_crossentropy, metrics=['accuracy'])

In [5]:
model.fit(X, y, epochs=1000, verbose=0)  

loss, accuracy = model.evaluate(X, y, verbose=0)
print("Evaluation Metrics : \n")
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

predictions = model.predict(X,verbose=0)
print("Predictions : ",np.round(predictions).astype(int).flatten())

Evaluation Metrics : 

Loss: 0.4294479191303253
Accuracy: 1.0
Predictions :  [0 1 1 0]
